In [ ]:
!pip install huggingface_hub datasets

# Make sure to apply for access to the llama3 models on huggingface(all llama family models require it). After that create a personal access token on [hugging face](https://huggingface.co/settings/tokens).

In [ ]:
!huggingface-cli login

In [ ]:
# check if gpu works
!nvidia-smi

In [2]:
import os
from huggingface_hub import snapshot_download

In [3]:
os.makedirs("my_results", exist_ok=True)

In [ ]:
!git clone https://github.com/XingyuHu109/LLM-Pruner.git

In [ ]:
!pip install -r LLM-Pruner/requirement.txt

In [ ]:
snapshot_download(repo_id="Neooooo/cs7643_models", local_dir="./LLM-Pruner")

In [33]:
# Choosing tasks to review
tasks = ["openbookqa", "arc_easy", "winogrande", "hellaswag", "arc_challenge", "piqa", "boolq"]
tasks = ",".join(tasks)

In [36]:
path_to_pruned_weights = "prune_log/vanilla_llama_1b_prune_0.25/pytorch_model.bin"
path_to_tuned_weights = "tune_log/llama3_1b_0.25_tune"
original_model = "meta-llama/Llama-3.2-1B-Instruct"
report_name = "llama3_1b_0.25_benchmark.json"

In [38]:
os.environ['PATH_TO_PRUNED_WEIGHTS'] = path_to_pruned_weights
os.environ['PATH_TO_TUNED_WEIGHTS'] = path_to_tuned_weights
os.environ['ORIGINAL_MODEL'] = original_model
os.environ['TASKS'] = tasks
os.environ['REPORT_NAME'] = report_name

In [ ]:
%%bash
export PYTHONPATH='./LLM-Pruner'
export HF_DATASETS_TRUST_REMOTE_CODE=true
python LLM-Pruner/lm-evaluation-harness/main.py --model hf-causal-experimental \
       --model_args "checkpoint=$PATH_TO_PRUNED_WEIGHTS,peft=$PATH_TO_TUNED_WEIGHTS,config_pretrained=$ORIGINAL_MODEL" \
       --tasks "$TASKS" \
       --device cuda:0 --no_cache \
       --output_path "../my_results/$REPORT_NAME"

In [22]:
# Additionally test speed-up and resource usage